
#  the model in this file comes from this website

#  https://geertlitjens.nl/post/getting-started-with-camelyon/



#  without editing the model, it is 80% accurate/confident

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

pcam, pcam_info = tfds.load('patch_camelyon', shuffle_files=True, with_info=True,
                            data_dir='/content/drive/My Drive/Colab Notebooks')

In [0]:
# Import NumPy to handle array's and Matplotlib for plotting loss curves
import numpy as np
import matplotlib.pyplot as plt

# Import TensorFlow and relevant Keras classes to setup the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation, LayerNormalization, Cropping2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [26]:
#just some network parameters, see above link regarding the layers for details
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

#dropout is used for regularization here with a probability of 0.3 for conv layers, 0.5 for the dense layer at the end
dropout_conv = 0.3
dropout_dense = 0.5

#initialize the model
model = Sequential()

#now add layers to it

#conv block 1
model.add(Conv2D(first_filters, kernel_size, input_shape = (96, 96, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))



#conv block 2
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))


#conv block 3
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))


#a fully connected (also called dense) layer at the end
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))

model.add(Dense(2, activation = "softmax"))

model.compile(optimizer=Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 94, 94, 32)        320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 94, 94, 32)        128       
_________________________________________________________________
activation_7 (Activation)    (None, 94, 94, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 92, 92, 32)        9216      
_________________________________________________________________
batch_normalization_8 (Batch (None, 92, 92, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 92, 92, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 46, 46, 32)       

In [0]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import array_ops
def rgb_to_grayscale2(images, name=None):
  """Converts one or more images from RGB to Grayscale.
  Outputs a tensor of the same `DType` and rank as `images`.  The size of the
  last dimension of the output is 1, containing the Grayscale value of the
  pixels.
  >>> original = tf.constant([[[1.0, 2.0, 3.0]]])
  >>> converted = tf.image.rgb_to_grayscale(original)
  >>> print(converted.numpy())
  [[[1.81...]]]
  Args:
    images: The RGB tensor to convert. The last dimension must have size 3 and
      should contain RGB values.
    name: A name for the operation (optional).
  Returns:
    The converted grayscale image(s).
  """
  with ops.name_scope(name, 'rgb_to_grayscale2', [images]) as name:
    images = ops.convert_to_tensor(images, name='images')
    # Remember original dtype to so we can convert back if needed
    orig_dtype = images.dtype
    flt_image = tf.image.convert_image_dtype(images, dtypes.float32)

    # Reference for converting between RGB and grayscale.
    # https://en.wikipedia.org/wiki/Luma_%28video%29
    rgb_weights = [0.2989, 0.5870, 0.1140]
    flt_image = math_ops.multiply(flt_image,flt_image)
    gray_float = math_ops.tensordot(flt_image, rgb_weights, [-1, -1])
    gray_float = tf.math.rsqrt(gray_float)
    gray_float = array_ops.expand_dims(gray_float, -1)
    return tf.image.convert_image_dtype(gray_float, orig_dtype, name=name)

In [0]:
import random

def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_jpeg_quality(image)
    image = tf.image.rot90(image, random.randint(0,3))
    image = tf.image.random_flip_left_right(image)
    

    image = tf.image.rgb_to_grayscale(image)
    #image2 = tf.image.rgb_to_hsv(image)
    #image = tf.image.central_crop(image,.5)
    #image2 = rgb_to_grayscale2(image)
    #image = tf.concat([image1,image2], axis = 2)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

In [0]:
train_pipeline = pcam['train'].map(convert_sample,
                                   num_parallel_calls=8).shuffle(1024).repeat().batch(64).prefetch(2)
valid_pipeline = pcam['validation'].map(convert_sample,
                                        num_parallel_calls=8).repeat().batch(128).prefetch(2)

In [30]:
hist = model.fit(train_pipeline,
                 validation_data=valid_pipeline,
                 verbose=2, epochs=25, steps_per_epoch=256, validation_steps=256)

Epoch 1/25
256/256 - 29s - loss: 0.5659 - accuracy: 0.7463 - val_loss: 1.3842 - val_accuracy: 0.5223
Epoch 2/25
256/256 - 34s - loss: 0.4885 - accuracy: 0.7820 - val_loss: 1.0009 - val_accuracy: 0.6659
Epoch 3/25
256/256 - 32s - loss: 0.4598 - accuracy: 0.7963 - val_loss: 0.4628 - val_accuracy: 0.7916
Epoch 4/25
256/256 - 38s - loss: 0.4562 - accuracy: 0.7943 - val_loss: 0.4589 - val_accuracy: 0.7866
Epoch 5/25
256/256 - 33s - loss: 0.4510 - accuracy: 0.7950 - val_loss: 0.5067 - val_accuracy: 0.7811
Epoch 6/25
256/256 - 33s - loss: 0.4551 - accuracy: 0.7964 - val_loss: 0.5950 - val_accuracy: 0.6942
Epoch 7/25
256/256 - 28s - loss: 0.4356 - accuracy: 0.8040 - val_loss: 0.4448 - val_accuracy: 0.7894
Epoch 8/25
256/256 - 31s - loss: 0.4194 - accuracy: 0.8165 - val_loss: 0.4130 - val_accuracy: 0.8120
Epoch 9/25
256/256 - 31s - loss: 0.4114 - accuracy: 0.8176 - val_loss: 0.4544 - val_accuracy: 0.7878
Epoch 10/25
256/256 - 31s - loss: 0.3942 - accuracy: 0.8292 - val_loss: 0.4207 - val_accura

In [33]:
test_pipeline = pcam['test'].map(convert_sample, num_parallel_calls=8).batch(128).prefetch(2)
print("Test set accuracy is {0:.4f}".format(model.evaluate(test_pipeline, steps=128, verbose=0)[1]))

Test set accuracy is 0.8583


In [0]:
#model.save("./patchcamelyon.hf5")